(sec:pandas10min_2)=
# 판다스 10분 완성 2부


**필수 라이브러리**

In [1]:
import numpy as np
import pandas as pd

## 합병과 결합: merge-join-concat

- 참고: [Merging section](https://pandas.pydata.org/docs/user_guide/merging.html#merging)

### 종/횡 결합: `pd.concat()` 함수

`pd.concat()` 함수는 여러 개의 데이터프레임을 하나로 합친다.

- `axis=0`: 종 결합. 즉 데이터프레임 여러 개의 위아래 결합.

In [2]:
df1 = pd.DataFrame(
    {
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    },
    index=[0, 1, 2, 3],
)

df1

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


In [3]:
df2 = pd.DataFrame(
    {
        "A": ["A4", "A5", "A6", "A7"],
        "B": ["B4", "B5", "B6", "B7"],
        "C": ["C4", "C5", "C6", "C7"],
        "D": ["D4", "D5", "D6", "D7"],
    },
    index=[4, 5, 6, 7],
)

df2

,A,B,C,D
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7


In [4]:
df3 = pd.DataFrame(
    {
        "A": ["A8", "A9", "A10", "A11"],
        "B": ["B8", "B9", "B10", "B11"],
        "C": ["C8", "C9", "C10", "C11"],
        "D": ["D8", "D9", "D10", "D11"],
    },
    index=[8, 9, 10, 11],
)

df3

,A,B,C,D
8,A8,B8,C8,D8
9,A9,B9,C9,D9
10,A10,B10,C10,D10
11,A11,B11,C11,D11


In [5]:
pd.concat([df1, df2, df3]) # axis=0 이 기본값

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7
8,A8,B8,C8,D8
9,A9,B9,C9,D9


- `axis=1`: 횡 결합. 즉 데이터프레임 여러 개의 좌우 결합.

In [6]:
df4 = pd.DataFrame(
    {
        "B": ["B2", "B3", "B6", "B7"],
        "D": ["D2", "D3", "D6", "D7"],
        "F": ["F2", "F3", "F6", "F7"],
    },
    index=[2, 3, 6, 7],
)

df4

,B,D,F
2,B2,D2,F2
3,B3,D3,F3
6,B6,D6,F6
7,B7,D7,F7


In [7]:
pd.concat([df1, df4], axis=1)

,A,B,C,D,B,D,F
0,A0,B0,C0,D0,NaN,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN,NaN
2,A2,B2,C2,D2,B2,D2,F2
3,A3,B3,C3,D3,B3,D3,F3
6,NaN,NaN,NaN,NaN,B6,D6,F6
7,NaN,NaN,NaN,NaN,B7,D7,F7


인덱스를 기존의 데이터프레임과 통일시키기 위해 리인덱싱을 활용할 수도 있다.

In [8]:
df1.index

Int64Index([0, 1, 2, 3], dtype='int64')

In [9]:
pd.concat([df1, df4], axis=1).reindex(df1.index)

,A,B,C,D,B,D,F
0,A0,B0,C0,D0,NaN,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN,NaN
2,A2,B2,C2,D2,B2,D2,F2
3,A3,B3,C3,D3,B3,D3,F3


In [10]:
pd.concat([df1, df4.reindex(df1.index)], axis=1)

,A,B,C,D,B,D,F
0,A0,B0,C0,D0,NaN,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN,NaN
2,A2,B2,C2,D2,B2,D2,F2
3,A3,B3,C3,D3,B3,D3,F3


### 합병: `pd.merge()` 함수

`pd.merge()` 함수 는 SQL 방식으로 특정 열을 기준으로 두 개의 데이터프레임을 합친다.
다양한 옵션을 지원하는 매우 강력한 도구이다.

- 참고: [Database style joining](https://pandas.pydata.org/docs/user_guide/merging.html#merging-join)

**예제**

실습을 위해 아래 두 데이터프레임을 이용한다.

In [11]:
left = pd.DataFrame({"key": ["foo", "foo"], "lval": [1, 2]})
right = pd.DataFrame({"key": ["foo", "foo"], "rval": [4, 5]})

In [12]:
left

,key,lval
0,foo,1
1,foo,2


In [13]:
right

,key,rval
0,foo,4
1,foo,5


- `on="key"` 키워드 인자
    - `key` 열에 사용된 항목 각각에 대해 다른 열에서 해당 항목과 연관된 값들을 조합할 수 있는 모든 경우의 수를 다룬다.
    - `foo` 값에 대해 `lval` 열에서 2개의 값이,
        `rval` 열에서 2개의 값이 있기에 `foo`와 관련해서 총 4개의 경우가 생성된다.
    <br><br>
    
    | `key` | `left.lval` | `right.rval` | 경우의 수 |
    | :---: | :---: | :---: | :---: |
    | `foo` | `1, 2` | `4, 5` | 4 |

In [14]:
pd.merge(left, right, on="key")

,key,lval,rval
0,foo,1,4
1,foo,1,5
2,foo,2,4
3,foo,2,5


**예제**

In [15]:
left = pd.DataFrame({"key": ["foo", "bar"], "lval": [1, 2]})
right = pd.DataFrame({"key": ["foo", "bar"], "rval": [4, 5]})

In [16]:
left

,key,lval
0,foo,1
1,bar,2


In [17]:
right

,key,rval
0,foo,4
1,bar,5


- `on="key"` 키워드 인자
    - `key` 열에 사용된 항목별로 모든 경우의 수를 다룬다.
    - `foo` 값에 대해 `lval` 열에서 1개의 값이,
        `rval` 열에서 1개의 값이 있기에 `foo`와 관련해서 총 1개의 경우가 생성된다.
    - `bar` 값에 대해 `lval` 열에서 1개의 값이,
        `rval` 열에서 1개의 값이 있기에 `foo`와 관련해서 총 1개의 경우가 생성된다.
    <br><br>
    
    | `key` | `left.lval` | `right.rval` | 경우의 수 |
    | :---: | :---: | :---: | :---: |
    | `foo` | `1` | `4` | 1 |        
    | `bar` | `2` | `5` | 1 |        

In [18]:
pd.merge(left, right, on="key")

,key,lval,rval
0,foo,1,4
1,bar,2,5


**예제**

경우의 수는 지정된 열의 항목이 사용된 횟수를 기준으로 한다. 

In [19]:
left = pd.DataFrame(
    {
        "key": ["K0", "K1", "K2", "K3"],
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
    }
)

In [20]:
left

,key,A,B
0,K0,A0,B0
1,K1,A1,B1
2,K2,A2,B2
3,K3,A3,B3


In [21]:
right = pd.DataFrame(
    {
        "key": ["K0", "K1", "K2", "K3"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    }
)

In [22]:
right

,key,C,D
0,K0,C0,D0
1,K1,C1,D1
2,K2,C2,D2
3,K3,C3,D3


| `key` | (`left.A`, `left.B`) | (`right.C`, `right.D`) | 경우의 수 |
| :---: | :---: | :---: | :---: |
| `K0` | (`A0`, `B0`) | (`C0`, `D0`) | 1 |
| `K1` | (`A1`, `B1`) | (`C1`, `D1`) | 1 |
| `K2` | (`A2`, `B2`) | (`C2`, `D2`) | 1 |
| `K3` | (`A3`, `B3`) | (`C3`, `D3`) | 1 |

In [23]:
result = pd.merge(left, right, on="key")
result

,key,A,B,C,D
0,K0,A0,B0,C0,D0
1,K1,A1,B1,C1,D1
2,K2,A2,B2,C2,D2
3,K3,A3,B3,C3,D3


**다양한 키 활용**

- 두 개 이상의 키를 하나의 쌍으로 된 키를 사용하는 경우와 유사함. 

In [24]:
left = pd.DataFrame(
    {
        "key1": ["K0", "K0", "K1", "K2"],
        "key2": ["K0", "K1", "K0", "K1"],
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
    }
)

left

,key1,key2,A,B
0,K0,K0,A0,B0
1,K0,K1,A1,B1
2,K1,K0,A2,B2
3,K2,K1,A3,B3


In [25]:
right = pd.DataFrame(
    {
        "key1": ["K0", "K1", "K1", "K2"],
        "key2": ["K0", "K0", "K0", "K0"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    }
)

right

,key1,key2,C,D
0,K0,K0,C0,D0
1,K1,K0,C1,D1
2,K1,K0,C2,D2
3,K2,K0,C3,D3


- `how='inner'`: 지정된 키의 교집합 대상

In [26]:
result = pd.merge(left, right, on=["key1", "key2"]) # how='inner' 가 기본값
result

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K1,K0,A2,B2,C1,D1
2,K1,K0,A2,B2,C2,D2


In [27]:
result = pd.merge(left, right, how="inner", on=["key1", "key2"])
result

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K1,K0,A2,B2,C1,D1
2,K1,K0,A2,B2,C2,D2


- `how='outer'`: 지정된 키의 합집합 대상

In [28]:
result = pd.merge(left, right, how="outer", on=["key1", "key2"])
result

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K0,K1,A1,B1,NaN,NaN
2,K1,K0,A2,B2,C1,D1
3,K1,K0,A2,B2,C2,D2
4,K2,K1,A3,B3,NaN,NaN
5,K2,K0,NaN,NaN,C3,D3


- `how='left'`: 왼쪽 데이터프레임의 키에 포함된 항목만 대상

In [29]:
left

,key1,key2,A,B
0,K0,K0,A0,B0
1,K0,K1,A1,B1
2,K1,K0,A2,B2
3,K2,K1,A3,B3


In [30]:
result = pd.merge(left, right, how="left", on=["key1", "key2"])
result

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K0,K1,A1,B1,NaN,NaN
2,K1,K0,A2,B2,C1,D1
3,K1,K0,A2,B2,C2,D2
4,K2,K1,A3,B3,NaN,NaN


- `how='right'`: 오른쪽 데이터프레임의 키에 포함된 항목만 대상

In [31]:
right

,key1,key2,C,D
0,K0,K0,C0,D0
1,K1,K0,C1,D1
2,K1,K0,C2,D2
3,K2,K0,C3,D3


In [32]:
result = pd.merge(left, right, how="right", on=["key1", "key2"])
result

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K1,K0,A2,B2,C1,D1
2,K1,K0,A2,B2,C2,D2
3,K2,K0,NaN,NaN,C3,D3


- `how='cross'`: 모든 경우의 수 조합

In [33]:
result = pd.merge(left, right, how="cross")
result

,key1_x,key2_x,A,B,key1_y,key2_y,C,D
0,K0,K0,A0,B0,K0,K0,C0,D0
1,K0,K0,A0,B0,K1,K0,C1,D1
2,K0,K0,A0,B0,K1,K0,C2,D2
3,K0,K0,A0,B0,K2,K0,C3,D3
4,K0,K1,A1,B1,K0,K0,C0,D0
5,K0,K1,A1,B1,K1,K0,C1,D1
6,K0,K1,A1,B1,K1,K0,C2,D2
7,K0,K1,A1,B1,K2,K0,C3,D3
8,K1,K0,A2,B2,K0,K0,C0,D0
9,K1,K0,A2,B2,K1,K0,C1,D1


### 합병: `DataFrame.join()` 메서드

인덱스를 기준으로 두 개의 데이터프레임을 합병할 때 사용한다.

In [34]:
left = pd.DataFrame(
    {"A": ["A0", "A1", "A2"], "B": ["B0", "B1", "B2"]}, index=["K0", "K1", "K2"]
)

left

,A,B
K0,A0,B0
K1,A1,B1
K2,A2,B2


In [35]:
right = pd.DataFrame(
    {"C": ["C0", "C2", "C3"], "D": ["D0", "D2", "D3"]}, index=["K0", "K2", "K3"]
)

right

,C,D
K0,C0,D0
K2,C2,D2
K3,C3,D3


In [36]:
left.join(right)

,A,B,C,D
K0,A0,B0,C0,D0
K1,A1,B1,NaN,NaN
K2,A2,B2,C2,D2


아래와 같이 `pd.merge()` 함수를 이용한 결과와 동일하다.

In [37]:
pd.merge(left, right, left_index=True, right_index=True, how='left')

,A,B,C,D
K0,A0,B0,C0,D0
K1,A1,B1,NaN,NaN
K2,A2,B2,C2,D2


`pd.merge()` 함수의 키워드 인자를 동일하게 사용할 수 있다.

- `how='outer'`

In [38]:
left.join(right, how="outer")

,A,B,C,D
K0,A0,B0,C0,D0
K1,A1,B1,NaN,NaN
K2,A2,B2,C2,D2
K3,NaN,NaN,C3,D3


아래 코드가 동일한 결과를 낸다.

In [39]:
pd.merge(left, right, left_index=True, right_index=True, how='outer')

,A,B,C,D
K0,A0,B0,C0,D0
K1,A1,B1,NaN,NaN
K2,A2,B2,C2,D2
K3,NaN,NaN,C3,D3


- `how='inner'`

In [40]:
left.join(right, how="inner")

,A,B,C,D
K0,A0,B0,C0,D0
K2,A2,B2,C2,D2


아래 코드가 동일한 결과를 낸다.

In [41]:
pd.merge(left, right, left_index=True, right_index=True, how='inner')

,A,B,C,D
K0,A0,B0,C0,D0
K2,A2,B2,C2,D2


## 다중 인덱스<font size='2'>MultiIndex</font>

- 참고: [Multiindex / advanced indexing](https://pandas.pydata.org/docs/user_guide/advanced.html)

다중 인덱스를 이용하여 데이터를 보다 체계적으로 다를 수 있다.
또한 이어서 다룰 그룹 분류<font size='2'>Group by</font>, 
모양 변환<font size='2'>reshaping</font>, 
피벗 변환<font size='2'>pivoting</font> 등에서 유용하게 활용된다.

### `MultiIndex` 객체

다중 인덱스 객체는 보통 튜플을 이용한다.
예를 들어 아래 두 개의 리스트를 이용하여 튜플을 생성한 다음 다중 인덱스로 만들어보자.

In [42]:
arrays = [
    ["bar", "bar", "baz", "baz", "foo", "foo", "qux", "qux"],
    ["one", "two", "one", "two", "one", "two", "one", "two"],
]

- 튜플 생성: 항목 8개

In [43]:
tuples = list(zip(*arrays))
tuples

[('bar', 'one'),
 ('bar', 'two'),
 ('baz', 'one'),
 ('baz', 'two'),
 ('foo', 'one'),
 ('foo', 'two'),
 ('qux', 'one'),
 ('qux', 'two')]

**다중 인덱스 객체 생성: `from_tupes()` 함수**

튜플 리스트를 이용하여 다중 인덱스 객체를 생성할 수 있다.

In [44]:
index = pd.MultiIndex.from_tuples(tuples)
index

MultiIndex([('bar', 'one'),
            ('bar', 'two'),
            ('baz', 'one'),
            ('baz', 'two'),
            ('foo', 'one'),
            ('foo', 'two'),
            ('qux', 'one'),
            ('qux', 'two')],
           )

- `names` 키워드 인자
    - 다중 인덱스의 각 레벨<font size='2'>level</font>의 이름 지정. 
    - 지정되지 않으면 `None`으로 처리됨.

예를 들어 위 코드에서 사용된 각각의 레벨에 이름은 다음과 같다.

- `"first"`: 0-레벨 이름
- `"second"`: 1-레벨 이름

In [45]:
index = pd.MultiIndex.from_tuples(tuples, names=["first", "second"])
index

MultiIndex([('bar', 'one'),
            ('bar', 'two'),
            ('baz', 'one'),
            ('baz', 'two'),
            ('foo', 'one'),
            ('foo', 'two'),
            ('qux', 'one'),
            ('qux', 'two')],
           names=['first', 'second'])

**다중 인덱스 객체 생성: `from_arrays()` 함수**

길이가 동일한 여러 개의 리스트로 구성된 어레이를 직접 이용할 수도 있다.

In [46]:
index = pd.MultiIndex.from_arrays(arrays, names=["first", "second"])
index

MultiIndex([('bar', 'one'),
            ('bar', 'two'),
            ('baz', 'one'),
            ('baz', 'two'),
            ('foo', 'one'),
            ('foo', 'two'),
            ('qux', 'one'),
            ('qux', 'two')],
           names=['first', 'second'])

### 다중 인덱스 라벨<font size='2'>label</font>을 사용하는 시리즈/데이터프레임 객체

- 시리즈 생성

아래 코드는 길이가 8인 어레이를 이용하여 시리즈를 생성한다.
인덱스의 라벨은 다중 인덱스가 사용된다.
각각의 레벨에서 라벨이 연속적으로 사용되는 경우는 보다 자연스러운 표현을 위해 생략되기도 한다.

In [47]:
s = pd.Series(np.random.randn(8), index=index)
s

first  second
bar    one       0.101908
       two      -0.091839
baz    one       0.165818
       two      -0.306306
foo    one      -2.019493
       two       0.126008
qux    one      -0.434825
       two      -0.775300
dtype: float64

- 데이터프레임 생성

아래 코드는 8개의 행으로 이뤄진 2차원 어레이를 이용하여 데이터프레임을 생성한다.
`index` 또는 `columns`로 여러 개의 리스트로 구성된 어레이를 지정하면
자동으로 다중 인덱스 라벨이 지정된다.

In [48]:
df = pd.DataFrame(np.random.randn(8, 4), index=arrays)
df

0         1         2         3
bar one -0.144317  0.889642 -0.624127 -0.319006
    two -0.782942  1.072020  0.713454  0.268914
baz one -0.941905  0.462984 -0.571175 -0.193206
    two -0.169682  1.638987  0.743565  1.521594
foo one  0.391385  0.636081 -0.068984 -2.357607
    two  2.035319  0.220498  0.591509  1.667710
qux one -0.730899 -0.403517  0.616465  1.112488
    two  0.551200 -1.005822  0.545140 -0.234190

다중 인덱스를 열 라벨로도 활용할 수 있다.
아래 코드는 8개의 열로 이뤄진 2차원 어레이를 이용하여 데이터프레임을 생성한다.

In [49]:
df1 = pd.DataFrame(np.random.randn(3, 8), index=["A", "B", "C"], columns=index)
df1

first        bar                 baz                 foo                 qux  \
second       one       two       one       two       one       two       one   
A       1.523679 -1.408990  0.624598 -0.449816 -1.347200 -0.843464  1.207034   
B       0.634362 -1.431597 -2.173192  0.149706 -0.387351  0.876548  0.933275   
C      -1.725519  1.376759  1.477540 -0.164795 -0.479897 -0.030103 -0.937097   

first             
second       two  
A       1.359975  
B       0.809387  
C      -2.102803

인덱스 라벨과 열 라벨 모두 다중 인덱스를 이용할 수도 있다.

- 동일한 길이의 리스트로 이루어진 리스트를 인덱스 또는 열의 라벨로 지정하면
    다중 인덱스로 자동 지정된다.

In [50]:
arrays2 = [
    ["toto", "toto", "titi", "titi", "tata", "tata"],
    ["A", "B", "A", "B", "A", "B"],
]

In [51]:
pd.DataFrame(np.random.randn(6, 6), index=index[:6], columns=arrays2)

toto                titi                tata          
                     A         B         A         B         A         B
first second                                                            
bar   one     0.318295 -0.391414  0.235857 -0.591067 -0.551706  0.140740
      two    -0.938588 -2.297387 -1.261008  0.221395 -1.551427 -0.122007
baz   one    -1.534477  0.528581  0.819568 -1.623191 -0.298260  0.879755
      two     0.561706 -0.213719 -0.602823  0.980408  0.231922  0.330181
foo   one    -0.915742 -0.747196 -0.708476 -0.789349 -0.257197 -0.022332
      two     0.489779 -0.889686  0.336396 -1.659560 -1.652459  0.268684

**주의사항**

튜플을 라벨로 사용하는 것은 다중 인덱스와 아무 상관 없다.
단지 라벨이 튜플인 것 뿐이다.

In [52]:
tuples

[('bar', 'one'),
 ('bar', 'two'),
 ('baz', 'one'),
 ('baz', 'two'),
 ('foo', 'one'),
 ('foo', 'two'),
 ('qux', 'one'),
 ('qux', 'two')]

In [53]:
pd.Series(np.random.randn(8), index=tuples)

(bar, one)   -0.564048
(bar, two)    0.046602
(baz, one)   -0.383040
(baz, two)   -1.771820
(foo, one)   -0.529103
(foo, two)    2.037847
(qux, one)   -0.347932
(qux, two)    0.899790
dtype: float64

### 인덱스의 레벨

다중 인덱스 객체의 `get_level_values()` 메서드를 이용하여 레벨별 인덱스 라벨을 확인할 수 있다.

- 0-레블 라벨

In [54]:
index.get_level_values(0)

Index(['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'], dtype='object', name='first')

레벨 이름을 이용할 수도 있다.

In [55]:
index.get_level_values("first")

Index(['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'], dtype='object', name='first')

- 1-레블 라벨

In [56]:
index.get_level_values(1)

Index(['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two'], dtype='object', name='second')

In [57]:
index.get_level_values("second")

Index(['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two'], dtype='object', name='second')

### 인덱싱

다중 인덱스를 라벨로 사용하는 시리즈와 데이터프레임의 인덱싱은 일반 인덱싱과 크게 다르지 않다.

- 시리즈 인덱싱

In [58]:
s

first  second
bar    one       0.101908
       two      -0.091839
baz    one       0.165818
       two      -0.306306
foo    one      -2.019493
       two       0.126008
qux    one      -0.434825
       two      -0.775300
dtype: float64

In [59]:
s["qux"]

second
one   -0.434825
two   -0.775300
dtype: float64

- 데이터프레임 인덱싱

In [60]:
df

0         1         2         3
bar one -0.144317  0.889642 -0.624127 -0.319006
    two -0.782942  1.072020  0.713454  0.268914
baz one -0.941905  0.462984 -0.571175 -0.193206
    two -0.169682  1.638987  0.743565  1.521594
foo one  0.391385  0.636081 -0.068984 -2.357607
    two  2.035319  0.220498  0.591509  1.667710
qux one -0.730899 -0.403517  0.616465  1.112488
    two  0.551200 -1.005822  0.545140 -0.234190

In [61]:
df.loc["bar"]

,0,1,2,3
one,-0.144317,0.889642,-0.624127,-0.319006
two,-0.782942,1.072020,0.713454,0.268914


레벨별로 라벨을 지정할 수 있다. 각각의 라벨은 쉼표로 구분한다.

In [62]:
df.loc["bar", "one"]

0   -0.144317
1    0.889642
2   -0.624127
3   -0.319006
Name: (bar, one), dtype: float64

아래와 같이 할 수도 있다.

In [63]:
df.loc["bar"].loc["one"]

0   -0.144317
1    0.889642
2   -0.624127
3   -0.319006
Name: one, dtype: float64

- 데이터프레임 인덱싱: 열 라벨이 다중 인덱스인 경우

In [64]:
df1

first        bar                 baz                 foo                 qux  \
second       one       two       one       two       one       two       one   
A       1.523679 -1.408990  0.624598 -0.449816 -1.347200 -0.843464  1.207034   
B       0.634362 -1.431597 -2.173192  0.149706 -0.387351  0.876548  0.933275   
C      -1.725519  1.376759  1.477540 -0.164795 -0.479897 -0.030103 -0.937097   

first             
second       two  
A       1.359975  
B       0.809387  
C      -2.102803

In [65]:
df1["bar"]

second,one,two
A,1.523679,-1.408990
B,0.634362,-1.431597
C,-1.725519,1.376759


레벨별로 라벨을 지정한다. 각각의 라벨은 쉼표로 구분한다.

In [66]:
df1["bar", "one"]

A    1.523679
B    0.634362
C   -1.725519
Name: (bar, one), dtype: float64

아래와 같이 할 수도 있다

In [67]:
df1["bar"]["one"]

A    1.523679
B    0.634362
C   -1.725519
Name: one, dtype: float64

### 슬라이싱

다중 인덱스를 라벨로 사용하는 시리즈와 데이터프레임의 인덱싱은 일반 슬라이싱과 크게 다르지 않다.

In [68]:
df

0         1         2         3
bar one -0.144317  0.889642 -0.624127 -0.319006
    two -0.782942  1.072020  0.713454  0.268914
baz one -0.941905  0.462984 -0.571175 -0.193206
    two -0.169682  1.638987  0.743565  1.521594
foo one  0.391385  0.636081 -0.068984 -2.357607
    two  2.035319  0.220498  0.591509  1.667710
qux one -0.730899 -0.403517  0.616465  1.112488
    two  0.551200 -1.005822  0.545140 -0.234190

- 0-레벨 인덱싱

In [69]:
df.loc["baz":"foo"]

0         1         2         3
baz one -0.941905  0.462984 -0.571175 -0.193206
    two -0.169682  1.638987  0.743565  1.521594
foo one  0.391385  0.636081 -0.068984 -2.357607
    two  2.035319  0.220498  0.591509  1.667710

- (0, 1)-레벨 인덱싱

In [70]:
df.loc[("baz", "two"):("qux", "one")]

0         1         2         3
baz two -0.169682  1.638987  0.743565  1.521594
foo one  0.391385  0.636081 -0.068984 -2.357607
    two  2.035319  0.220498  0.591509  1.667710
qux one -0.730899 -0.403517  0.616465  1.112488

튜플들의 리스트를 지정하면 리인덱싱처럼 작동한다.

In [71]:
df.loc[[("bar", "two"), ("qux", "one")]]

,,0,1,2,3
bar,two,-0.782942,1.072020,0.713454,0.268914
qux,one,-0.730899,-0.403517,0.616465,1.112488


이외에 `slice()` 함수와 `pd.IndexSlice` 객체를 사용하는 방법도 있지만 여기서는 다루지 않는다.

## 그룹 분류: `pd.groupby()` 함수

- 참고: [Grouping section](https://pandas.pydata.org/docs/user_guide/groupby.html#groupby)

`pd.groupby()` 함수는 다음 3 기능을 제공한다.

- **분류**<font size='2'>Splitting</font>: 데이터를 조건에 따라 여러 그룹으로 분류
- **함수 적용**<font size='2'>Applying</font>: 그룹별로 함수 적용
- **조합**<font size='2'>Combining</font>: 그룹별 함수 적용 결과를 취합하여 새로운 데이터프레임/시리즈 생성

In [72]:
df = pd.DataFrame({'A': ['foo', 'bar', 'foo', 'bar',
                         'foo', 'bar', 'foo', 'bar'],
                   'B': ['one', 'one', 'two', 'three',
                         'two', 'two', 'one', 'three'],
                   'C': np.random.randn(8),
                   'D': np.random.randn(8)})

df

,A,B,C,D
0,foo,one,-2.096667,-0.316257
1,bar,one,-1.050753,-0.003154
2,foo,two,0.022887,-0.502613
3,bar,three,-0.275926,-0.545020
4,foo,two,-1.096496,0.210605
5,bar,two,-1.725943,0.771117
6,foo,one,-0.556452,0.012967
7,bar,three,-2.082001,0.290652


- `A` 열에 사용된 항목 기준으로 그룹으로 분류한 후 그룹별로 `C`와 `D` 열의 모든 항목의 합 계산해서 새로운 데이터프레임 생성
    <br><br>
    
    | `A` | 경우의 수 |
    | :---: | :---: |
    | `bar` | 1 |
    | `foo` | 1 |

In [73]:
df.groupby('A')[["C", "D"]].sum()

,C,D
A,,
bar,-5.134624,0.513595
foo,-3.726728,-0.595299


- `A`열의 항목과 `B` 열의 항목의 조합을 기준으로 그룹으로 그룹별로 `C`와 `D` 열의 모든 항목의 합 계산해서 새로운 데이터프레임 생성
    <br><br>
    
    | `A` | `B` | 경우의 수 |
    | :---: | :---: | :---: |
    | `bar` | `one`, `three`, `two` | 3 |
    | `foo` | `one`, `two` | 2 |

In [74]:
df.groupby(["A", "B"]).sum()

C         D
A   B                        
bar one   -1.050753 -0.003154
    three -2.357927 -0.254368
    two   -1.725943  0.771117
foo one   -2.653119 -0.303291
    two   -1.073609 -0.292008

**그룹 확인**

- `for` 반복문 활용 

In [75]:
for name, group in df.groupby(["A", "B"]):
    print(name)
    print(group)

('bar', 'one')
     A    B         C         D
1  bar  one -1.050753 -0.003154
('bar', 'three')
     A      B         C         D
3  bar  three -0.275926 -0.545020
7  bar  three -2.082001  0.290652
('bar', 'two')
     A    B         C         D
5  bar  two -1.725943  0.771117
('foo', 'one')
     A    B         C         D
0  foo  one -2.096667 -0.316257
6  foo  one -0.556452  0.012967
('foo', 'two')
     A    B         C         D
2  foo  two  0.022887 -0.502613
4  foo  two -1.096496  0.210605


- `get_group()` 메서드

In [76]:
df.groupby(["A", "B"]).get_group(('bar', 'one'))

,A,B,C,D
1,bar,one,-1.050753,-0.003154


In [77]:
df.groupby(["A", "B"]).get_group(('bar', 'three'))

,A,B,C,D
3,bar,three,-0.275926,-0.545020
7,bar,three,-2.082001,0.290652


- `groups` 속성

In [78]:
df.groupby(["A", "B"]).groups

{('bar', 'one'): [1], ('bar', 'three'): [3, 7], ('bar', 'two'): [5], ('foo', 'one'): [0, 6], ('foo', 'two'): [2, 4]}

- `value_counts` 속성

In [79]:
df.groupby(["A", "B"]).value_counts()

A    B      C          D        
bar  one    -1.050753  -0.003154    1
     three  -2.082001   0.290652    1
            -0.275926  -0.545020    1
     two    -1.725943   0.771117    1
foo  one    -2.096667  -0.316257    1
            -0.556452   0.012967    1
     two    -1.096496   0.210605    1
             0.022887  -0.502613    1
dtype: int64

- `nunique` 속성

In [80]:
df.groupby(["A", "B"]).nunique()

C  D
A   B          
bar one    1  1
    three  2  2
    two    1  1
foo one    2  2
    two    2  2

- `sort=True` 키워드 인자

In [81]:
df.groupby(["A", "B"], sort=True).sum()

C         D
A   B                        
bar one   -1.050753 -0.003154
    three -2.357927 -0.254368
    two   -1.725943  0.771117
foo one   -2.653119 -0.303291
    two   -1.073609 -0.292008

In [82]:
df.groupby(["A", "B"], sort=False).sum()

C         D
A   B                        
foo one   -2.653119 -0.303291
bar one   -1.050753 -0.003154
foo two   -1.073609 -0.292008
bar three -2.357927 -0.254368
    two   -1.725943  0.771117

In [83]:
df.groupby(["A", "B"], sort=False).nunique()

C  D
A   B          
foo one    2  2
bar one    1  1
foo two    2  2
bar three  2  2
    two    1  1

**그룹 연산 예제**

- `max()` 메서드

In [84]:
df.groupby('A')[["C", "D"]].max()

,C,D
A,,
bar,-0.275926,0.771117
foo,0.022887,0.210605


In [85]:
df.groupby(["A", "B"]).max()

C         D
A   B                        
bar one   -1.050753 -0.003154
    three -0.275926  0.290652
    two   -1.725943  0.771117
foo one   -0.556452  0.012967
    two    0.022887  0.210605

- `mean()` 메서드

In [86]:
df.groupby('A')[["C", "D"]].mean()

,C,D
A,,
bar,-1.283656,0.128399
foo,-0.931682,-0.148825


In [87]:
df.groupby(["A", "B"]).mean()

C         D
A   B                        
bar one   -1.050753 -0.003154
    three -1.178964 -0.127184
    two   -1.725943  0.771117
foo one   -1.326560 -0.151645
    two   -0.536805 -0.146004

- `size()` 메서드

In [88]:
df.groupby('A')[["C", "D"]].size()

A
bar    4
foo    4
dtype: int64

In [89]:
df.groupby(["A", "B"]).size()

A    B    
bar  one      1
     three    2
     two      1
foo  one      2
     two      2
dtype: int64

- `describe()` 메서드

In [90]:
df.groupby('A')[["C", "D"]].describe()

C                                                              \
    count      mean       std       min       25%       50%       75%   
A                                                                       
bar   4.0 -1.283656  0.796395 -2.082001 -1.814958 -1.388348 -0.857046   
foo   4.0 -0.931682  0.901176 -2.096667 -1.346539 -0.826474 -0.411618   

                  D                                                    \
          max count      mean       std       min       25%       50%   
A                                                                       
bar -0.275926   4.0  0.128399  0.550821 -0.545020 -0.138620  0.143749   
foo  0.022887   4.0 -0.148825  0.320710 -0.502613 -0.362846 -0.151645   

                         
          75%       max  
A                        
bar  0.410768  0.771117  
foo  0.062376  0.210605

In [91]:
df.groupby(["A", "B"]).describe()

C                                                              \
          count      mean       std       min       25%       50%       75%   
A   B                                                                         
bar one     1.0 -1.050753       NaN -1.050753 -1.050753 -1.050753 -1.050753   
    three   2.0 -1.178964  1.277088 -2.082001 -1.630483 -1.178964 -0.727445   
    two     1.0 -1.725943       NaN -1.725943 -1.725943 -1.725943 -1.725943   
foo one     2.0 -1.326560  1.089096 -2.096667 -1.711613 -1.326560 -0.941506   
    two     2.0 -0.536805  0.791523 -1.096496 -0.816650 -0.536805 -0.256959   

                        D                                                    \
                max count      mean       std       min       25%       50%   
A   B                                                                         
bar one   -1.050753   1.0 -0.003154       NaN -0.003154 -0.003154 -0.003154   
    three -0.275926   2.0 -0.127184  0.590909 -0.545020 -0.336102 -0.127184   
    two   -1.725943   1.0  0.771117       NaN  0.771117  0.771117  0.771117   
foo one   -0.556452   2.0 -0.151645  0.232797 -0.316257 -0.233951 -0.151645   
    two    0.022887   2.0 -0.146004  0.504321 -0.502613 -0.324308 -0.146004   

                               
                75%       max  
A   B                          
bar one   -0.003154 -0.003154  
    three  0.081734  0.290652  
    two    0.771117  0.771117  
foo one   -0.069339  0.012967  
    two    0.032300  0.210605

## 모양 변환<font size='2'>Reshaping</font>

### 항목 재배열

- 참고: [Reshaping](https://pandas.pydata.org/docs/user_guide/reshaping.html#reshaping-stacking)

**스택**

열 인덱스의 레벨을 하나 줄일 때 사용한다.
없어진 레벨은 행 인덱스의 마지막 레벨로 추가된다.

In [92]:
index

MultiIndex([('bar', 'one'),
            ('bar', 'two'),
            ('baz', 'one'),
            ('baz', 'two'),
            ('foo', 'one'),
            ('foo', 'two'),
            ('qux', 'one'),
            ('qux', 'two')],
           names=['first', 'second'])

In [93]:
df = pd.DataFrame(np.random.randn(8, 2), index=index, columns=["A", "B"])
df

A         B
first second                    
bar   one     0.310946  1.030382
      two    -2.916903  1.153140
baz   one    -0.860177 -0.516968
      two    -0.531007  0.421749
foo   one     0.817539 -1.231142
      two     1.896280 -0.572233
qux   one    -1.749213  1.787404
      two     0.512943 -0.036622

In [94]:
df2 = df[:4]
df2

A         B
first second                    
bar   one     0.310946  1.030382
      two    -2.916903  1.153140
baz   one    -0.860177 -0.516968
      two    -0.531007  0.421749

- `stack()` 메서드: 
    열이 한 개의 레벨로 구성되어 있기에 `stack()` 메서드를 적용하면
    결국 모든 열이 없어지고, 열의 라벨은
    인덱스의 마지막 레벨의 라벨로 변환된다.
    여기서는 결국 3중 인덱스를 사용하는 시리즈를 생성한다.

In [95]:
stacked = df2.stack()
stacked

first  second   
bar    one     A    0.310946
               B    1.030382
       two     A   -2.916903
               B    1.153140
baz    one     A   -0.860177
               B   -0.516968
       two     A   -0.531007
               B    0.421749
dtype: float64

**언스택**

행 인덱스의 지정된 레벨을 열의 마지막 레벨로 변환한다.
인자를 지정하지 않으면 마지막 레벨을 변환한다.

- `unstack()` 메서드

In [96]:
stacked.unstack()

A         B
first second                    
bar   one     0.310946  1.030382
      two    -2.916903  1.153140
baz   one    -0.860177 -0.516968
      two    -0.531007  0.421749

In [97]:
stacked.unstack().unstack()

A                   B          
second       one       two       one       two
first                                         
bar     0.310946 -2.916903  1.030382  1.153140
baz    -0.860177 -0.531007 -0.516968  0.421749

인자를 지정하면 해당 레벨을 열의 마지막 레벨로 변환한다.

In [98]:
stacked.unstack(0)

first          bar       baz
second                      
one    A  0.310946 -0.860177
       B  1.030382 -0.516968
two    A -2.916903 -0.531007
       B  1.153140  0.421749

In [99]:
stacked.unstack(1)

second        one       two
first                      
bar   A  0.310946 -2.916903
      B  1.030382  1.153140
baz   A -0.860177 -0.531007
      B -0.516968  0.421749

### 피버팅

- 참고: [Pivot Tables](https://pandas.pydata.org/docs/user_guide/reshaping.html#reshaping-pivot)

**`pd.pivot_table()` 함수**

**예제**

In [100]:
import datetime

df = pd.DataFrame(
    {
        "A": ["one", "one", "two", "three"] * 6,
        "B": ["A", "B", "C"] * 8,
        "C": ["foo", "foo", "foo", "bar", "bar", "bar"] * 4,
        "D": np.random.randn(24),
        "E": np.random.randn(24),
    }
)

df

,A,B,C,D,E
0,one,A,foo,-0.341797,0.349000
1,one,B,foo,-0.976225,0.091472
2,two,C,foo,-1.255961,-0.636587
3,three,A,bar,0.919116,-2.305688
4,one,B,bar,1.442856,1.140716
5,one,C,bar,-0.396454,-0.598928
6,two,A,foo,1.456405,1.079286
7,three,B,foo,-1.114995,0.734640
8,one,C,foo,0.465591,-1.453113
9,one,A,bar,-0.528248,-3.012864


In [101]:
pd.pivot_table(df, values="D", index=["A", "B"], columns=["C"]) # aggfunc=np.mean 이 기본값

C             bar       foo
A     B                    
one   A  0.400884 -0.427561
      B  1.465319 -0.905390
      C -0.028989  0.386806
three A  1.051737       NaN
      B       NaN -0.615730
      C -0.154699       NaN
two   A       NaN  0.903528
      B  0.784886       NaN
      C       NaN -0.112492

In [102]:
pd.pivot_table(df, values="D", index=["A", "B"], columns=["C"], aggfunc=np.sum)

C             bar       foo
A     B                    
one   A  0.801767 -0.855122
      B  2.930638 -1.810780
      C -0.057978  0.773613
three A  2.103474       NaN
      B       NaN -1.231459
      C -0.309397       NaN
two   A       NaN  1.807057
      B  1.569771       NaN
      C       NaN -0.224984

**`DataFrame.pivot()` 메서드**

In [103]:
df

,A,B,C,D,E
0,one,A,foo,-0.341797,0.349000
1,one,B,foo,-0.976225,0.091472
2,two,C,foo,-1.255961,-0.636587
3,three,A,bar,0.919116,-2.305688
4,one,B,bar,1.442856,1.140716
5,one,C,bar,-0.396454,-0.598928
6,two,A,foo,1.456405,1.079286
7,three,B,foo,-1.114995,0.734640
8,one,C,foo,0.465591,-1.453113
9,one,A,bar,-0.528248,-3.012864


In [104]:
df1 = df.groupby(['A', 'B', 'C']).sum().reset_index()
df1

,A,B,C,D,E
0,one,A,bar,0.801767,-2.794749
1,one,A,foo,-0.855122,-0.177582
2,one,B,bar,2.930638,2.605526
3,one,B,foo,-1.810780,0.078715
4,one,C,bar,-0.057978,-0.456534
5,one,C,foo,0.773613,-3.186938
6,three,A,bar,2.103474,-1.600055
7,three,B,foo,-1.231459,1.004115
8,three,C,bar,-0.309397,1.724540
9,two,A,foo,1.807057,2.197040


In [105]:
df1.pivot(index=['A', 'B'], columns='C', values="D")

C             bar       foo
A     B                    
one   A  0.801767 -0.855122
      B  2.930638 -1.810780
      C -0.057978  0.773613
three A  2.103474       NaN
      B       NaN -1.231459
      C -0.309397       NaN
two   A       NaN  1.807057
      B  1.569771       NaN
      C       NaN -0.224984

## 시계열

시계열<font size='2'>time series</font> 데이터는 시간의 흐름을 다루는 데이터다. 증시, 날씨 데이터 등에서 많이 사용된다.
판다스 라이브러리는 시계열 데이터를 다루는 많은 기능을 제공한다.

- 참고: [Time Series](https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries)

**`pd.date_range()` 함수**

시간을 기준으로 시간 인덱스 자료형을 생성한다.
아래 코드에 사용된 키워드 인자의 의미는 다음과 같다.

- `periods=100`: 첫째 인자로 지정된 시간부터 100개의 시간 데이터 샘플 생성
- `freq="S"`: 100개의 시간 데이터 샘플을 초 단위로 생성

In [106]:
rng = pd.date_range("1/1/2023", periods=100, freq="S")
rng

DatetimeIndex(['2023-01-01 00:00:00', '2023-01-01 00:00:01',
               '2023-01-01 00:00:02', '2023-01-01 00:00:03',
               '2023-01-01 00:00:04', '2023-01-01 00:00:05',
               '2023-01-01 00:00:06', '2023-01-01 00:00:07',
               '2023-01-01 00:00:08', '2023-01-01 00:00:09',
               '2023-01-01 00:00:10', '2023-01-01 00:00:11',
               '2023-01-01 00:00:12', '2023-01-01 00:00:13',
               '2023-01-01 00:00:14', '2023-01-01 00:00:15',
               '2023-01-01 00:00:16', '2023-01-01 00:00:17',
               '2023-01-01 00:00:18', '2023-01-01 00:00:19',
               '2023-01-01 00:00:20', '2023-01-01 00:00:21',
               '2023-01-01 00:00:22', '2023-01-01 00:00:23',
               '2023-01-01 00:00:24', '2023-01-01 00:00:25',
               '2023-01-01 00:00:26', '2023-01-01 00:00:27',
               '2023-01-01 00:00:28', '2023-01-01 00:00:29',
               '2023-01-01 00:00:30', '2023-01-01 00:00:31',
               '2023-01-

아래 코드는 0부터 500 사이에서 100개의 정수를 임의로 생성한 다음에 
위 시간 인덱스를 이용하여 시리즈를 생성한다.

In [107]:
ts = pd.Series(np.random.randint(0, 500, len(rng)), index=rng)
ts

2023-01-01 00:00:00    281
2023-01-01 00:00:01      7
2023-01-01 00:00:02     79
2023-01-01 00:00:03    252
2023-01-01 00:00:04    269
                      ... 
2023-01-01 00:01:35     39
2023-01-01 00:01:36    126
2023-01-01 00:01:37    381
2023-01-01 00:01:38    320
2023-01-01 00:01:39    200
Freq: S, Length: 100, dtype: int64

**`resample()` 메서드**

 인자로 지정된 기준으로 구룹 생성 후 집계<font size='2'>aggregation</font> 실행한다.

아래 코드의 의미는 다음과 같다.
- `"10S"`: 10초 단위로 데이터를 그룹으로 묶기
- `sum()` 메서도: 그룹별로 속한 데이터 샘플 모두 더하기

In [108]:
ts.resample("10S").sum()

2023-01-01 00:00:00    1304
2023-01-01 00:00:10    1859
2023-01-01 00:00:20    3091
2023-01-01 00:00:30    1702
2023-01-01 00:00:40    2234
2023-01-01 00:00:50    2507
2023-01-01 00:01:00    2660
2023-01-01 00:01:10    2614
2023-01-01 00:01:20    2859
2023-01-01 00:01:30    2560
Freq: 10S, dtype: int64

아래 코드의 의미는 다음과 같다.
- `"1Min"`: 1분 단위로 데이터를 그룹으로 묶기
- `sum()` 메서도: 그룹별로 속한 데이터 샘플 모두 더하기

In [109]:
ts.resample("1Min").sum()

2023-01-01 00:00:00    12697
2023-01-01 00:01:00    10693
Freq: T, dtype: int64

**시간 대 기간**

시간<font size='2'>time stamp</font>과 기간<font size='2'>period</font>의 차이점과
상호 변환 방식을 살펴본다.

- 2023년 1월 1일부터 월 단위로 12개의 항목을 갖는 시간 인덱스 생성

In [110]:
rng = pd.date_range("1/1/2023", periods=12, freq="M")
rng

DatetimeIndex(['2023-01-31', '2023-02-28', '2023-03-31', '2023-04-30',
               '2023-05-31', '2023-06-30', '2023-07-31', '2023-08-31',
               '2023-09-30', '2023-10-31', '2023-11-30', '2023-12-31'],
              dtype='datetime64[ns]', freq='M')

- 앞서 생성된 시간 인덱스를 라벨로 사용하는 시리즈 생성

In [111]:
ts = pd.Series(np.random.randn(len(rng)), index=rng)
ts

2023-01-31   -0.048456
2023-02-28   -0.437135
2023-03-31   -0.858183
2023-04-30   -1.464642
2023-05-31   -0.682526
2023-06-30   -0.054873
2023-07-31   -0.247072
2023-08-31    0.899058
2023-09-30   -0.141416
2023-10-31   -1.732241
2023-11-30   -0.831102
2023-12-31   -0.197133
Freq: M, dtype: float64

**`to_period() 메서드`**

- 시간 인덱스를 월 단위의 기간 인덱스로 변환

In [112]:
ps = ts.to_period()
ps

2023-01   -0.048456
2023-02   -0.437135
2023-03   -0.858183
2023-04   -1.464642
2023-05   -0.682526
2023-06   -0.054873
2023-07   -0.247072
2023-08    0.899058
2023-09   -0.141416
2023-10   -1.732241
2023-11   -0.831102
2023-12   -0.197133
Freq: M, dtype: float64

**`to_timestamp() 메서드`**

- 기간 인덱스를 월 단위의 시간 인덱스로 변환

In [113]:
ps.to_timestamp()

2023-01-01   -0.048456
2023-02-01   -0.437135
2023-03-01   -0.858183
2023-04-01   -1.464642
2023-05-01   -0.682526
2023-06-01   -0.054873
2023-07-01   -0.247072
2023-08-01    0.899058
2023-09-01   -0.141416
2023-10-01   -1.732241
2023-11-01   -0.831102
2023-12-01   -0.197133
Freq: MS, dtype: float64

## 데이터셋 불러오기와 저장하기

**csv 파일 불러오기**

- 참고: [`DataFrame.to_csv()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html#pandas.DataFrame.to_csv)

In [114]:
df.to_csv("foo.csv")

**csv 파일로 저장하기**

- 참고: [`read_csv()`](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html#pandas.read_csv)

In [115]:
pd.read_csv("foo.csv")

,Unnamed: 0,A,B,C,D,E
0,0,one,A,foo,-0.341797,0.349000
1,1,one,B,foo,-0.976225,0.091472
2,2,two,C,foo,-1.255961,-0.636587
3,3,three,A,bar,0.919116,-2.305688
4,4,one,B,bar,1.442856,1.140716
5,5,one,C,bar,-0.396454,-0.598928
6,6,two,A,foo,1.456405,1.079286
7,7,three,B,foo,-1.114995,0.734640
8,8,one,C,foo,0.465591,-1.453113
9,9,one,A,bar,-0.528248,-3.012864


**엑셀 파일 불러오기**

- 참고: [`DataFrame.to_excel()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_excel.html#pandas.DataFrame.to_excel)

In [116]:
df.to_excel("foo.xlsx", sheet_name="Sheet1")

**엑셀 파일로 저장하기**

- 참고: [`read_excel()`](https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html#pandas.read_excel)

In [117]:
pd.read_excel("foo.xlsx", "Sheet1", index_col=None, na_values=["NA"])

,Unnamed: 0,A,B,C,D,E
0,0,one,A,foo,-0.341797,0.349000
1,1,one,B,foo,-0.976225,0.091472
2,2,two,C,foo,-1.255961,-0.636587
3,3,three,A,bar,0.919116,-2.305688
4,4,one,B,bar,1.442856,1.140716
5,5,one,C,bar,-0.396454,-0.598928
6,6,two,A,foo,1.456405,1.079286
7,7,three,B,foo,-1.114995,0.734640
8,8,one,C,foo,0.465591,-1.453113
9,9,one,A,bar,-0.528248,-3.012864
